In [1]:
!pip install -q together
!pip install -q FlagEmbedding
!pip install -q peft
!pip install -q faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.9/347.9 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.6 MB/s eta 0:00:00

In [2]:
from google.colab import userdata
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')

In [3]:
from FlagEmbedding import FlagModel
from FlagEmbedding import FlagReranker

import os
import torch
import faiss
import re
import json
import numpy as np
import requests

from together import Together

In [4]:
together = Together(api_key=TOGETHER_API_KEY)

In [5]:
# encode_queries()和encode()的区别：
# for s2p (short query to long passage) retrieval task, suggest to use encode_queries() which will automatically add the instruction to each query
# corpus in retrieval task can still use encode() or encode_corpus(), since they don't need instruction

model = FlagModel('BAAI/bge-base-en-v1.5', query_instruction_for_retrieval="Represent this sentence for searching relevant passages:", use_fp16=True)
# print(model.device)

reranker = FlagReranker('BAAI/bge-reranker-base', use_fp16=True, device='cuda')
# print(reranker.device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [24]:
# query = "Tell me the exam information for the database class."
query = "What is the whether in Beijing?"

courses_numbers = ['544', '566', '585', '596', '599', '626', '677', '699']
course_names = ['Applied Natural Language Processing (NLP)', 'Deep Learning and its Applications (DL)', 'Database Systems (database)', 'Scientific Computing and Visualization',
         'Distributed Systems', 'Text as Data', 'Advanced Computer Vision (CV)', 'Robotic Perception (Robotics)']

full_course_info = dict(zip(['CSCI' + num for num in courses_numbers], ['CSCI' + num + ' ' + name for num, name in zip(courses_numbers, course_names)]))

name_to_num = dict(zip(course_names, courses_numbers))

In [7]:
full_course_info

{'CSCI544': 'CSCI544 Applied Natural Language Processing (NLP)',
 'CSCI566': 'CSCI566 Deep Learning and its Applications (DL)',
 'CSCI585': 'CSCI585 Database Systems (database)',
 'CSCI596': 'CSCI596 Scientific Computing and Visualization',
 'CSCI599': 'CSCI599 Distributed Systems',
 'CSCI626': 'CSCI626 Text as Data',
 'CSCI677': 'CSCI677 Advanced Computer Vision (CV)',
 'CSCI699': 'CSCI699 Robotic Perception (Robotics)'}

In [8]:
def build_vector_database(course_names):
    course_embeddings = model.encode(course_names)
    np_course_embeddings = np.array(course_embeddings).astype('float32')

    # with normalization -> bacome cosine similarity search
    faiss.normalize_L2(np_course_embeddings)

    # IndexFlatIP: dot-product search, IndexFlatL2: L2 search
    index_innerproduct = faiss.IndexFlatIP(len(course_embeddings[0]))
    res = faiss.StandardGpuResources()  # Create GPU resources
    gpu_index = faiss.index_cpu_to_gpu(res, 0, index_innerproduct) # Move index to GPU

    gpu_index.add(np_course_embeddings)

    return gpu_index

database = build_vector_database(course_names)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [9]:
def search_(query, database, embed_dim, topk=1):
    query_embed = model.encode_queries(query)
    query_embed = np.array(query_embed).astype('float32')

    _, idx = database.search(query_embed.reshape((1, embed_dim)), topk) # (768,) -> (1, 768)
    idx = idx.reshape(-1)

    ret = [course_names[i] for i in idx]

    return ret

def find_syllabus(query, courses_numbers, course_names, name_to_num, model, database):
    def find_course():
        for num in courses_numbers:
            if num in query:
                return num

        result = search_(query, database, embed_dim=database.d, topk=1)
        return result[0]

    result = find_course()

    if result in courses_numbers:
        return 'CSCI' + result + '.txt'
    else:
        return 'CSCI' + name_to_num[result] + '.txt'

In [10]:
print(find_syllabus(query, courses_numbers, course_names, name_to_num, model, database))

CSCI585.txt


In [11]:
# 读取txt文件并按"\n\n"切割 -> 按照需求自行修改！！！
def read_and_split_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    segments = content.split('##')
    # print(segments)

    return [seg.strip() for seg in segments]  # 变成了一个list

# 计算query和每个文档分段的相似度，并返回排序后的结果
def compute_similarity(query, segments):
    # encode_queries是专门用于encode query的
    query_embedding = model.encode_queries(query)  # [512, ]
    segment_embeddings = model.encode(segments)  # [num_seg, 512]

    # 官方提供的。只计算点积，不计算cos-simi
    similarities = [query_embedding @ segment_embedding.T for segment_embedding in segment_embeddings]

    # 将分数、query和文档分段组合并排序 -> 降序
    results = sorted(zip(similarities, segments), key=lambda x: x[0], reverse=True)
    # print(results)

    return results

# 定义生成查询和文档组合的函数
def generate_query_passage_pairs(query, passages):
    return [[query[0], passage] for passage in passages]

In [12]:
def RAG(**kwargs):
    query = kwargs.get('query', None)
    if query is None:
        raise ValueError('Your query cannot be empty!')

    k = 3

    final_syllabus = find_syllabus(query, courses_numbers, course_names, name_to_num, model, database)
    print(final_syllabus)

    file_path = '/content' + os.sep + final_syllabus
    knowledge_base = read_and_split_file(file_path)

    query = [query]
    all_similarity = compute_similarity(query, knowledge_base)

    results = []
    for score, segment in all_similarity:
        if score > 0.25:
            results.append(segment)

    query_passage_pairs = generate_query_passage_pairs(query, results)

    if query_passage_pairs:
        scores = reranker.compute_score(query_passage_pairs)
        sorted_query_passage_pairs = sorted(zip(scores, query_passage_pairs), key=lambda x: x[0], reverse=True)

        sorted_results = [(score, pair[1]) for score, pair in sorted_query_passage_pairs]

        final_results = []
        for score, passage in sorted_results:
            # if score > 0:
            final_results.append((score, passage))

        final_results = final_results[:k]
    else:
        final_results = [(0.0, 'There is no relevant information for the given query!')]

    all_segments = '\n\n'.join([result[1] for result in final_results])
    class_found = final_syllabus[:-len('.txt')]

    return f'Here are class information for {full_course_info[class_found]}:\n{all_segments}'

In [13]:
def financial_news_report(**kwargs):
    query = kwargs.get('query', None)
    if query is None:
        raise ValueError('Your query cannot be empty!')

    new_api_key = userdata.get('new_api')
    endpoint = 'https://newsapi.org/v2/everything'

    # Set query parameters
    params = {
        'q': 'financial OR stock market OR investment',  # Specify topics
        'language': 'en',     # Language preference
        'sortBy': 'publishedAt',  # Order by most recent
        'apiKey': new_api_key,
    }

    # Send request to News API
    response = requests.get(endpoint, params=params)

    # Check for successful response
    if response.status_code == 200:
        news_data = response.json()
        # Specify the maximum number of articles to retrieve
        max_articles = 5  # Set this to the number you want
        articles = news_data.get('articles', [])[:max_articles]  # Slice to limit the number
        fianl_res = ""
        # Parse and print each article's title and description
        for article in articles:
            print("Title:", article['title'])
            print("Description:", article['description'])
            fianl_res += "Title:" + article['title'] + "\n"
            fianl_res += "Description:" + article['description'] + "\n"

    else:
        print("Failed to retrieve news:", response.status_code)

    return fianl_res

In [14]:
def get_current_weather(**kwargs):
    # This would be replaced by a weather API
    # aa = f"What's the weather in {location}?"
    location = kwargs.get('location', None)
    if location is None:
        raise ValueError('Your location cannot be empty!')

    if location == "San Francisco, CA":
        return "The weather in San Francisco is 62 degrees and cloudy"
    elif location == "Philadelphia, PA":
        return "The weather in Philadelphia is 83 degrees and sunny"
    elif location == "Tokyo, Japan":
        return "The weather in Tokyo is 27 degrees and sunny"

    return f"The weather in {location} is unknown"

In [15]:
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "RAG",
            "description": "Retrieve the relevant section in the given knowledge base when the user asks information about courses or syllabus.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The original question that the user asks exactly, no need to rephrase.",
                    },
                },
                "required": ["query"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Fetches the current weather for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The name of the city and country (e.g., 'San Francisco, US').",
                    },
                },
                "required": ["location"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "financial_news_report",
            "description": "Retrieve recent financial news when the user asks for it.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The original question that the user asks exactly, no need to rephrase.",
                    },
                },
                "required": ["query"],
                "additionalProperties": False,
            },
        }
    },
]

In [25]:
toolPrompt = f"""
You have access to the following functions:

Use the function '{tools_list[0]['function']['name']}' to '{tools_list[0]['function']['description']}'.
The parameters are: {json.dumps(tools_list[0]['function']['parameters']['properties'])}, where {tools_list[0]['function']['parameters']['required']} are required.

Use the function '{tools_list[1]['function']['name']}' to '{tools_list[1]['function']['description']}':
The parameters are: {json.dumps(tools_list[1]['function']['parameters']['properties'])}, where {tools_list[1]['function']['parameters']['required']} are required.

Use the function '{tools_list[2]['function']['name']}' to '{tools_list[2]['function']['description']}':
The parameters are: {json.dumps(tools_list[2]['function']['parameters']['properties'])}, where {tools_list[2]['function']['parameters']['required']} are required.

If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=example_function_name>{{\"example_name\": \"example_value\"}}</function>

Reminder:
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- If there is no function call available, answer the question like normal with your current knowledge and do not tell the user about function calls

"""

messages = [
  	{
        "role": "system",
        "content": toolPrompt,
    },
    {
        "role": "user",
        "content": query,
    },

]

# functions = [tool["function"] for tool in tools_list]

response = together.chat.completions.create(
    model="Qwen/Qwen2.5-7B-Instruct-Turbo",
    messages=messages,
    max_tokens=1024,
    temperature=0,
    # functions=functions,
    # tools=tools_list,
    # tool_choice="auto",
)

messages.append(response.choices[0].message)
print(response.choices[0].message.content)

<function=get_current_weather>{"location": "Beijing, CN"} </function>


In [26]:
print(toolPrompt)


You have access to the following functions:

Use the function 'RAG' to 'Retrieve the relevant section in the given knowledge base when the user asks information about courses or syllabus.'.
The parameters are: {"query": {"type": "string", "description": "The original question that the user asks exactly, no need to rephrase."}}, where ['query'] are required.

Use the function 'get_current_weather' to 'Fetches the current weather for a given location.':
The parameters are: {"location": {"type": "string", "description": "The name of the city and country (e.g., 'San Francisco, US')."}}, where ['location'] are required.

Use the function 'financial_news_report' to 'Retrieve recent financial news when the user asks for it.':
The parameters are: {"query": {"type": "string", "description": "The original question that the user asks exactly, no need to rephrase."}}, where ['query'] are required.

If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<fun

In [27]:
print(response.choices[0].message)  # -> role=<MessageRole.ASSISTANT: 'assistant'> content=None tool_calls=[ToolCalls(id='call_5wbhh8zyrqa8hf98nw3lqfyu', type='function', function=FunctionCall(name='RAG', arguments='{"query":"csci585 final exam date"}'), index=0)]
# print(response.choices[0].message.tool_calls[0])  # one function at a time -> take index 0 -> id='call_5wbhh8zyrqa8hf98nw3lqfyu' type='function' function=FunctionCall(name='RAG', arguments='{"query":"csci585 final exam date"}') index=0
# print()
# print(response.choices[0].message.tool_calls[0].function)  # -> name='RAG' arguments='{"query":"csci585 final exam date"}'
# print()
# print(response.choices[0].message.tool_calls[0].function.name)  # Function name -> RAG
# print()
# print(response.choices[0].message.tool_calls[0].function.arguments)  # Function arguments -> {"query":"csci585 final exam date"}

role=<MessageRole.ASSISTANT: 'assistant'> content='<function=get_current_weather>{"location": "Beijing, CN"} </function>' tool_calls=[]


In [28]:
def parse_tool_response(response_message):
    """
    response_message: response.choices[0].message
    """
    if response_message.tool_calls:
        parsed_response = {
            "function": response_message.tool_calls[0].function.name,
            "arguments": json.loads(response_message.tool_calls[0].function.arguments)
          }

        return parsed_response

    function_regex = r"<function=([a-zA-Z_]\w*)>(\{.*?\})\s*(?:</function>|<function(?:/[\w]*)?>)"
    match = re.search(function_regex, response_message.content)

    if match:
        function_name, args_string = match.groups()
        args_string = args_string.strip()

        try:
            args = json.loads(args_string)

            parsed_response = {
                "function": function_name,
                "arguments": args,
            }

            return parsed_response

        except json.JSONDecodeError as error:
            print(f"Error parsing function arguments: {error}")
            return None

    return None

In [29]:
parsed_response = parse_tool_response(response.choices[0].message)

if parsed_response is not None:
    print('The model decide to call a function: ')
    print(parsed_response)
else:
    print("No function call found in the response")
    print(response.choices[0].message.content)

The model decide to call a function: 
{'function': 'get_current_weather', 'arguments': {'location': 'Beijing, CN'}}


In [30]:
if parsed_response:
    available_functions = {
        "RAG": RAG,
        "get_current_weather": get_current_weather,
        "financial_news_report": financial_news_report,
    }

    if parsed_response["function"] not in available_functions:
        available_function_names = "\n".join(available_functions.keys())
        raise NotImplementedError(
            f"Function {parsed_response['function']} is not implemented. "
            f"Our available functions are:\n\n{available_function_names}"
        )

    function_to_call = available_functions[parsed_response["function"]]

    # result, final_course = function_to_call(query)
    result = function_to_call(**parsed_response["arguments"])
    print(result)
    print()

    answer_prompt = "You are a very helpful assistant. Please answer user's question according to given information. Trust the given information, it is completely align with the user's question."

    new_messages = [
        {
            "role": "system",
            "content": answer_prompt,
        },
        {
            "role": "user",
            "content": f"""
## Question:
{query}

## Information:
{result}
"""
        },
    ]

    res = together.chat.completions.create(
        model="Qwen/Qwen2.5-7B-Instruct-Turbo",
        messages=new_messages,
        max_tokens=1000,
        temperature=0.9,
    )
    print("Answer from the LLM: ", res.choices[0].message)
else:
    print("No function call found in the response")

The weather in Beijing, CN is unknown

Answer from the LLM:  role=<MessageRole.ASSISTANT: 'assistant'> content="I'm sorry, but the information provided does not specify the current weather in Beijing. To get the most accurate and up-to-date weather information, you can check a reliable weather forecasting website or app." tool_calls=[]


In [31]:
print(answer_prompt)
print(new_messages[1]['content'])

You are a very helpful assistant. Please answer user's question according to given information. Trust the given information, it is completely align with the user's question.

## Question:
What is the whether in Beijing?

## Information:
The weather in Beijing, CN is unknown



In [32]:
print(res.choices[0].message.content)

I'm sorry, but the information provided does not specify the current weather in Beijing. To get the most accurate and up-to-date weather information, you can check a reliable weather forecasting website or app.
